<a href="https://colab.research.google.com/github/elisaboccini/AMDproject/blob/main/11_04IMDbSpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"elisaboccini","key":"3024605213579b61d22c8ab4775d25f4"}'}

In [2]:
#kaggle API 
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -q ashirwadsangwan/imdb-dataset
!unzip imdb-dataset.zip

kaggle.json
Archive:  imdb-dataset.zip
  inflating: name.basics.tsv.gz      
  inflating: name.basics.tsv/name.basics.tsv  
  inflating: title.akas.tsv.gz       
  inflating: title.akas.tsv/title.akas.tsv  
  inflating: title.basics.tsv.gz     
  inflating: title.basics.tsv/title.basics.tsv  
  inflating: title.principals.tsv.gz  
  inflating: title.principals.tsv/title.principals.tsv  
  inflating: title.ratings.tsv.gz    
  inflating: title.ratings.tsv/title.ratings.tsv  


In [3]:
#Spark session creation

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz 
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 49.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=1cab08861a0198d9313d77de6d4430301b0aeaaa3dc670140523e0ffdc5e8215
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [5]:
import findspark
findspark.init('spark-3.1.1-bin-hadoop2.7')

In [6]:
from pyspark.sql import SparkSession

In [7]:
#spark = SparkSession.builder.master("local[*]").getOrCreate()
#try with 4 g to avoid garbage collection
spark = SparkSession.builder.master("local")\
        .appName("MBA")\
        .config('spark.ui.port', '4050')\
        .config("spark.driver.memory", "15g") \
        .config("spark.driver.maxResultSize", "1g") \
        .config("spark.sql.broadcastTimeout", "10h") \
        .getOrCreate()

In [8]:
import pyspark
type(spark)

pyspark.sql.session.SparkSession

In [9]:
sc = spark.sparkContext

In [10]:
principals = spark.read.csv('title.principals.tsv.gz', sep="\t", header=True, inferSchema=True)
#akas = spark.read.csv('title.akas.tsv.gz', sep="\t", header=True, inferSchema=True)
titlebasics = spark.read.csv('title.basics.tsv.gz', sep="\t", header=True, inferSchema=True)
#ratings = spark.read.csv('title.ratings.tsv.gz', sep="\t", header=True, inferSchema=True)
namesbasics = spark.read.csv('name.basics.tsv.gz', sep="\t", header=True, inferSchema=True)

In [11]:
principals.createOrReplaceTempView("principals")
#akas.createOrReplaceTempView("akas")
titlebasics.createOrReplaceTempView("titlebasics")
#ratings.createOrReplaceTempView("ratings")
namesbasics.createOrReplaceTempView("namesbasics")

In [12]:
# read csv of principals, titlebasics and namesbasics
# drop duplicates in titlebasics
titlebasics.dropDuplicates((['tconst'])).count() # 6326545

DataFrame[tconst: string, titleType: string, primaryTitle: string, originalTitle: string, isAdult: int, startYear: string, endYear: string, runtimeMinutes: string, genres: string]

In [13]:
namesbasics.dropDuplicates((['nconst'])).count() #9711022

DataFrame[nconst: string, primaryName: string, birthYear: string, deathYear: string, primaryProfession: string, knownForTitles: string]

In [14]:
data1 = spark.sql('''SELECT tb.tconst, tb.primaryTitle, nb.primaryName, nb.nconst 
                          FROM titlebasics as tb LEFT JOIN principals as p ON tb.tconst = p.tconst INNER JOIN namesbasics as nb ON p.nconst = nb.nconst
                          WHERE p.category = 'actor' OR p.category = 'actress' ''')

data1.createOrReplaceTempView('data1')
data1.show(5)
data1.count() #14mln

+---------+-----------------+--------------+---------+
|   tconst|     primaryTitle|   primaryName|   nconst|
+---------+-----------------+--------------+---------+
|tt0083109| The Cabbage Soup|Louis de Funès|nm0000086|
|tt0062120|The Little Bather|Louis de Funès|nm0000086|
|tt0057422|    Squeak-squeak|Louis de Funès|nm0000086|
|tt0043691|     The Gamblers|Louis de Funès|nm0000086|
|tt0335439| Station mondaine|Louis de Funès|nm0000086|
+---------+-----------------+--------------+---------+
only showing top 5 rows



14811027

In [15]:
actor_movies_d1 = spark.sql(""" SELECT nconst, primaryName as actor, COUNT(tconst) as movies
                                FROM data1
                                GROUP BY nconst, primaryName
                                ORDER BY movies desc""")
actor_movies_d1.show(5) # the 5 most frequent actors -> movies are 6 mln and the most frequent appears in 5000 -> minSupport?
#I should plot how they distribute vd Bodini

+----------+------------------+------+
|    nconst|             actor|movies|
+----------+------------------+------+
|nm10120013|   Sameera Sherief|  5746|
| nm0151534|    Sudha Chandran|  4400|
| nm0192504|   Francisco Cuoco|  3987|
| nm0555829|     Claudio Marzo|  3917|
| nm0525123|Luz Stella Luengas|  3817|
+----------+------------------+------+
only showing top 5 rows



In [17]:
sns.set(rc={"figure.figsize":(18,7)})
ax = sns.histplot(data1.toPandas(), x = "actor", binwidth = 2, log_scale = (False, True))
ax.set(title = "Number of total purchased items per customer", xlabel = "Number of items", ylabel = "Number of customers");
ax.set_xticks(range(0,150,5));

KeyboardInterrupt: ignored

In [ ]:
#NEW
#generazione dei baskets
from pyspark.sql.functions import collect_set

moviesasbaskets_d1 = data1.groupBy('tconst','primaryTitle').agg(collect_set('primaryName').alias('cast')) #metti un alias
moviesasbaskets_d1.createOrReplaceTempView('moviesasbaskets_d1')

moviesasbaskets_d1.show(n=5)

+---------+--------------------+--------------------+
|   tconst|        primaryTitle|                cast|
+---------+--------------------+--------------------+
|tt0000007|Corbett and Court...|[James J. Corbett...|
|tt0000378|          Les vagues|[Julie Souplet, G...|
|tt0000621|   That Fatal Sneeze|[Gertie Potter, T...|
|tt0000653|A Calamitous Elop...|[John R. Cumpson,...|
|tt0000841|       The Day After|[Marion Leonard, ...|
+---------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
moviesasbaskets_d1.count() #3602120

3602120

In [ ]:
#concatenate using hyphen
# to have info about the title but still maintaining the basket form (so just one col for the basket and the corresponding
# list of itemsets)
from pyspark.sql import functions as f

moviesasbaskets_d1 = moviesasbaskets_d1.withColumn('movie', f.concat(f.col('tconst'),f.lit('-'),f.col('primaryTitle')))

moviesasbaskets_d1 = moviesasbaskets_d1.select('movie', 'cast')
moviesasbaskets_d1.show(5)

+--------------------+--------------------+
|               movie|                cast|
+--------------------+--------------------+
|tt0000007-Corbett...|[James J. Corbett...|
|tt0000378-Les vagues|[Julie Souplet, G...|
|tt0000621-That Fa...|[Gertie Potter, T...|
|tt0000653-A Calam...|[John R. Cumpson,...|
|tt0000841-The Day...|[Marion Leonard, ...|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
# Caching aka persisting-> otherwise randomsplit will lead to inconsistencies, since it changes the randomsplit each
# time you perform an operation on the splitted data
moviesasbaskets_d1 = moviesasbaskets_d1.persist(pyspark.StorageLevel.MEMORY_AND_DISK)

In [ ]:
# apply the randomSplit() 
MBAtrain,MBAtest = moviesasbaskets_d1.randomSplit([0.9, 0.1], seed=13)

In [ ]:
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="cast", minSupport=0.0001, minConfidence=0)
model_d1 = fpGrowth.fit(MBAtrain) #7 minutes

In [ ]:
# Display frequent itemsets.-> NB this result includes all size items!
from pyspark.sql.functions import col, asc,desc
freqItemsets_d1 = model_d1.freqItemsets
freqItemsets_d1.createOrReplaceTempView("freqItemsets_d1")
freqItemsets_d1.orderBy(col('freq').desc()).show() #the most frequent actor is nm10120013 appearing in > 5000 films.

+--------------------+----+
|               items|freq|
+--------------------+----+
|   [Sameera Sherief]|5199|
|    [Sudha Chandran]|4004|
|   [Francisco Cuoco]|3572|
|     [Claudio Marzo]|3515|
|[Luz Stella Luengas]|3456|
|        [Tony Ramos]|3384|
|[Luis Eduardo Motoa]|3359|
|   [Macdonald Carey]|3336|
|       [John Clarke]|3271|
|[Luis Eduardo Mot...|3222|
|     [Regina Duarte]|3214|
|        [Jim Hunter]|3170|
|       [Ray Meagher]|3135|
| [Rohini Hattangadi]|3067|
|[Caroline de Bruijn]|2979|
|    [Tarcísio Meira]|2965|
|        [Ravi Kiran]|2947|
|           [Mounica]|2947|
|[Mounica, Sameera...|2946|
|[Ravi Kiran, Moun...|2946|
+--------------------+----+
only showing top 20 rows



In [ ]:
# Display generated association rules.
associationRules_d1 = model_d1.associationRules
associationRules_d1.createOrReplaceTempView("associationRules_d1")
associationRules_d1.show(n=5)

+--------------------+-------------------+------------------+-----------------+--------------------+
|          antecedent|         consequent|        confidence|             lift|             support|
+--------------------+-------------------+------------------+-----------------+--------------------+
|[Rohini Banerjee,...|     [Shruti Bapna]|               1.0|6306.780155642024|1.178399505195600...|
|[Rohini Banerjee,...|[Aishwarya Sakhuja]|               1.0|5830.368705035971|1.178399505195600...|
|[Rohini Banerjee,...| [Darshan Jariwala]|0.9293193717277487|7531.401668848167|1.095109487812665...|
|[Rohini Banerjee,...|     [Shraman Jain]|0.9790575916230366|7229.604348292764|1.153720981526582...|
|[Rohini Banerjee,...|       [Ravi Dubey]|               1.0|2390.623156342183|1.178399505195600...|
+--------------------+-------------------+------------------+-----------------+--------------------+
only showing top 5 rows



In [ ]:
spark.sql("""select count (*)
           from associationRules_d1
           where confidence = '1.0' """).show() # I obtained 8969 association rules where confidence is 1
           #should I rise the minSupport?

+--------+
|count(1)|
+--------+
|    8547|
+--------+



In [ ]:
# transform examines the input items against all the association rules and summarize the consequents as prediction
# it would be useful to divide the db in training and test set in order to actually do predictions
associations_d1 = model_d1.transform(MBAtest)
associations_d1.show()

+--------------------+--------------------+----------+
|               movie|                cast|prediction|
+--------------------+--------------------+----------+
|tt0000378-Les vagues|[Julie Souplet, G...|        []|
|   tt0006235-The Vow|    [Marion Leonard]|        []|
|tt0006399-The Ban...|[Francis Ford, Jo...|        []|
|tt0006538-The Com...|[Harold Lockwood,...|        []|
|tt0007213-The Pri...|[Frank Whitson, D...|        []|
|tt0017047-The Win...|[Ivan Klyukvin, V...|        []|
|tt0017055-The Lad...|[Roy Stewart, Ral...|        []|
|tt0018691-Beware ...|[William Collier ...|        []|
|tt0020073-The Lad...|[Mona Goya, Bruce...|        []|
|tt0027147-Two for...|[Mary Boland, Bin...|        []|
|tt0028509-Windbag...|[Norma Varden, Gr...|        []|
|tt0029282-My Arti...|[Mae Questel, Jac...|        []|
|tt0030958-The Nov...|[Jean Galland, Je...|        []|
| tt0034335-Under Age|[Mary Anderson, T...|        []|
|tt0035213-Le prin...|[Jimmy Gaillard, ...|        []|
|tt0035788

In [ ]:
associations_d1.createOrReplaceTempView('associations_d1')
pred1= spark.sql("""select *
                      from associations_d1
                      where size(prediction) >= 1 """)
pred1.createOrReplaceTempView('pred1')
pred1.show()

+--------------------+--------------------+--------------------+
|               movie|                cast|          prediction|
+--------------------+--------------------+--------------------+
|tt0048313-The Loo...|[Thomas Gomez, Ju...|[Carolyn Jones, D...|
|tt0081337-A Piece...|[Jack Kirwan, Cha...|[Georgie Parker, ...|
|tt0148079-Doing W...|[Gwen Davies, Jac...|       [Mae Questel]|
| tt0212701-Triángulo|[Julieta Egurrola...|[Verónica Merchan...|
|tt0219102-SS Papa...|[Kostas Kazakos, ...|  [Katia Dandoulaki]|
|tt0227881-Forbidd...|[Arturo Bonín, Lu...|[Gastón Pauls, Be...|
|tt0340824-Lucien ...|[Oswald Maes, Jan...|[Chris Boni, Myri...|
|tt0377271-La torm...|[Jorge Mondragón,...|[Claudia Islas, J...|
|tt0391939-Dos gal...|[Armando Infante,...|[Andrea Bonelli, ...|
|tt0407521-Devilma...|[Atsuko Enomoto, ...|[Wasabi Mizuta, M...|
|tt0472911-Strong-...|[Carlos Morales, ...|[Angelene Aguilar...|
|tt0518111-A Sombr...|[Vítor Norte, Els...|[Rogério Samora, ...|
|tt0528988-Episode...|[Su

In [ ]:
pred1.count() ##55128 -> ora 55502 perchè?

55502

In [ ]:
#for a better understanding 
spark.sql("""select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence
              from associationRulesd1
              order by confidence desc
              limit 20""").show()

+--------------------+-------------------+----------+
|     antecedent (if)|  consequent (then)|confidence|
+--------------------+-------------------+----------+
|[Susanne Stuart, ...|  [Michael Beecher]|       1.0|
|[Abel Abelyan, S....| [Samvel Sarkisyan]|       1.0|
|[Fernando Farrés,...|   [Adolfo Chuiman]|       1.0|
|[Rohini Banerjee,...|[Aishwarya Sakhuja]|       1.0|
|[Fernando Farrés,...|   [Michel Finseth]|       1.0|
|[Susanne Stuart, ...|    [Judy McBurney]|       1.0|
|[Rohini Banerjee,...|     [Shruti Bapna]|       1.0|
|[Alejandra Azcára...|      [Kathy Sáenz]|       1.0|
|[David Mardyan, G...|   [Lika Salmanyan]|       1.0|
|[Rohini Banerjee,...|       [Ravi Dubey]|       1.0|
|[David Mardyan, H...|  [Aida Babajanyan]|       1.0|
|[Alfonso Burgos, ...| [Ezequiel Castaño]|       1.0|
|[David Mardyan, H...| [Luiza Ghambaryan]|       1.0|
|[Alfonso Burgos, ...| [Camila Bordonaba]|       1.0|
|[David Mardyan, H...|   [Lika Salmanyan]|       1.0|
|[David Mardyan, G...|   [Ar

In [ ]:
#focus the analysis for frequent itemset with one actor only (frequent actors)
#NOW filter freqItemsets by those having items = 1 -> most frequent actors banally
mostfrequentactor_d1= spark.sql("""select items, freq
                                    from freqItemsets_d1
                                    where size(items) = 1
                                    order by freq desc""")
mostfrequentactor_d1.show(n=5)
mostfrequentactor_d1.count()

+--------------------+----+
|               items|freq|
+--------------------+----+
|   [Sameera Sherief]|5199|
|    [Sudha Chandran]|4004|
|   [Francisco Cuoco]|3572|
|     [Claudio Marzo]|3515|
|[Luz Stella Luengas]|3456|
+--------------------+----+
only showing top 5 rows



4684

In [ ]:
#now analysis for frequent itemset with two actors
mostfrequent2actors_d1= spark.sql("""select items, freq
                                      from freqItemsets_d1
                                      where size(items) = 2
                                      order by freq desc""")
mostfrequent2actors_d1.show(n=5)
mostfrequent2actors_d1.count()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Luis Eduardo Mot...|3222|
|[Ravi Kiran, Moun...|2946|
|[Mounica, Sameera...|2946|
|[Ravi Kiran, Same...|2946|
|[Kavita Lad, Rohi...|2887|
+--------------------+----+
only showing top 5 rows



2658

In [ ]:
#now analysis for frequent itemset with three actors
mostfrequent3actors_d1= spark.sql("""select items, freq
                                      from freqItemsets_d1
                                      where size(items) = 3
                                      order by freq desc""")
mostfrequent3actors_d1.show(n=5)
mostfrequent3actors_d1.count()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Ravi Kiran, Moun...|2946|
|[Frances Reid, Jo...|2293|
|[Hari Teja, Subha...|2243|
|[Bhavya Gandhi, D...|2215|
|[Ana Velázquez, S...|2125|
+--------------------+----+
only showing top 5 rows



1713

In [ ]:
#Trying the same model but with a minsupport of 0.001 instead of 0.0001-> frequent itemsets are still generated since 0.001 of 3mln is 3000 and therefore there are actors playing in 3000 movies
#but in this way only 9 are considered frequent and no association rule and no predictions are generated
#we have to find a middle way

#with 0.0005 seems best option since it reduces association rules and predictions (1600 not null predictions) but it produces some

In [ ]:
#try if the association rules are correct: 
spark.sql("""select (*)
           from associationRules_d1
           where confidence = '1.0' 
           limit 20""").show()
#for example we obtain in this table
#|       [Jui Gadkari]|        [Astad Kale]|       1.0|1847.1139601139603|5.389172606221764E-4|
#|    [Gagan Chinappa]|       [Kavya Shree]|       1.0|1849.2213348545351|5.407681498973527E-4|
#|        [Neha Gowda]|           [Rachana]|       1.0|1989.9846531614487|5.009740304810615E-4|
#|        [Neha Gowda]|   [Branden Garrett]|       1.0|1983.8953488372092|5.009740304810615E-4|
#then we could select movies where [Jui Gadkari] played and check if in the cast there is also [Astad Kale]

+--------------------+--------------------+----------+------------------+--------------------+
|          antecedent|          consequent|confidence|              lift|             support|
+--------------------+--------------------+----------+------------------+--------------------+
|[Rohini Banerjee,...|      [Shruti Bapna]|       1.0| 6306.780155642024|1.178399505195600...|
|[Rohini Banerjee,...| [Aishwarya Sakhuja]|       1.0| 5830.368705035971|1.178399505195600...|
|[Rohini Banerjee,...|        [Ravi Dubey]|       1.0| 2390.623156342183|1.178399505195600...|
|[Alejandra Azcára...|       [Kathy Sáenz]|       1.0|3692.1241457858773|1.385082140923624...|
|[David Mardyan, H...|    [Arsen Levonyan]|       1.0| 7171.869469026548|1.391251771840879E-4|
|[David Mardyan, H...|   [Aida Babajanyan]|       1.0| 4838.335820895522|1.391251771840879E-4|
|[David Mardyan, H...|[Artashes Aleksan...|       1.0| 7031.854663774403|1.391251771840879E-4|
|[David Mardyan, H...|  [Luiza Ghambaryan]|       

In [ ]:
#NB episodes are included in this dataset and this is biasing the results as can be seen from the queries below: the most frequent itemsets and the 
# generated association rules are related not to movies but episodes -> it is very likely that the actors appearing in an episode will be playing
# also in the other episodes of the same TVseries

q1= ('''SELECT * 
        FROM data1
        WHERE primaryName = 'Jui Gadkari' ''')
spark.sql(q1).show()
#then take a random tconst and check if in the cast there is Astad Kale

#or also do this query to see in which films they appear to be together -> they are all episodes! 
#so the model is actually biased by the presence of all these episodes in which the actors always appear together

q2=('''SELECT *
        FROM data1 as d1 INNER JOIN data1 as d2 ON d1.tconst = d2.tconst
        WHERE d1.primaryName = 'Jui Gadkari' AND d2.primaryName = 'Astad Kale' ''')
spark.sql(q2).show()

#let's try with the second couple performing the same query as above
q3=('''SELECT *
        FROM data1 as d1 INNER JOIN explicit as d2 ON d1.tconst = d2.tconst
        WHERE d1.primaryName = 'Gagan Chinappa' AND d2.primaryName = 'Kavya Shree' ''')
spark.sql(q3).show()
#same situation: they act together in each episode -> quite normal

q4=('''SELECT *
        FROM data1 as d1 INNER JOIN data1 as d2 ON d1.tconst = d2.tconst
        WHERE d1.primaryName = 'Neha Gowda' AND d2.primaryName = 'Rachana' ''')
spark.sql(q4).show()

#so I will reperform the process by filtering the explicit dataset and considering only movies and cortos, not TV series

In [ ]:
#which are the types of items collected in the titlebasics db?
spark.sql('''SELECT distinct (titleType)
              FROM titlebasics''').show()

In [18]:
data2 = spark.sql('''SELECT tb.tconst, tb.primaryTitle, nb.primaryName, nb.nconst 
                          FROM titlebasics as tb LEFT JOIN principals as p ON tb.tconst = p.tconst INNER JOIN namesbasics as nb ON p.nconst = nb.nconst
                          WHERE ((p.category = 'actor' OR p.category = 'actress') AND (tb.titleType = 'movie' OR tb.titleType = 'tvMovie' OR tb.titleType = 'short' OR tb.titleType = 'tvShort')) ''')

data2.createOrReplaceTempView('data2')
data2.show(5)
data2.count() #3686467-> this also reduces a lot the size of the db (from 14 mln to 3 mln)

+---------+-----------------+--------------+---------+
|   tconst|     primaryTitle|   primaryName|   nconst|
+---------+-----------------+--------------+---------+
|tt0083109| The Cabbage Soup|Louis de Funès|nm0000086|
|tt0062120|The Little Bather|Louis de Funès|nm0000086|
|tt0057422|    Squeak-squeak|Louis de Funès|nm0000086|
|tt0043691|     The Gamblers|Louis de Funès|nm0000086|
|tt0335439| Station mondaine|Louis de Funès|nm0000086|
+---------+-----------------+--------------+---------+
only showing top 5 rows



3686467

In [ ]:
queryx= ('''select count(distinct tconst) 
            from data2''')
spark.sql(queryx).show() # 974512 -> we have actually 1/3 of the values we had before!

In [ ]:
#lets see if we can group by primarytitle (if the distinct titles are equivalent to n of tconst)
queryx2= ('''select count(distinct primaryTitle) 
            from data2''')
spark.sql(queryx2).show() #756111 -> situation hasn't changed: still there are homonyms in movies and shorts

In [ ]:
#generazione dei baskets
from pyspark.sql.functions import collect_set

moviesasbaskets_d2 = data2.groupBy('tconst','primaryTitle').agg(collect_set('primaryName').alias('cast')) #metti un alias
moviesasbaskets_d2.createOrReplaceTempView('moviesasbaskets_d2')

moviesasbaskets_d2.show(n=5) #974512

+---------+--------------------+--------------------+
|   tconst|        primaryTitle|                cast|
+---------+--------------------+--------------------+
|tt0000007|Corbett and Court...|[James J. Corbett...|
|tt0000378|          Les vagues|[Julie Souplet, G...|
|tt0000621|   That Fatal Sneeze|[Gertie Potter, T...|
|tt0000653|A Calamitous Elop...|[John R. Cumpson,...|
|tt0000841|       The Day After|[Marion Leonard, ...|
+---------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
#concatenate using hyphen
# to have info about the title but still maintaining the basket form (so just one col for the basket and the corresponding
# list of itemsets)
from pyspark.sql import functions as f

moviesasbaskets_d2 = moviesasbaskets_d2.withColumn('movie', f.concat(f.col('tconst'),f.lit('-'),f.col('primaryTitle')))

moviesasbaskets_d2 = moviesasbaskets_d2.select('movie', 'cast')
moviesasbaskets_d2.show(5)

+--------------------+--------------------+
|               movie|                cast|
+--------------------+--------------------+
|tt0000007-Corbett...|[James J. Corbett...|
|tt0000378-Les vagues|[Julie Souplet, G...|
|tt0000621-That Fa...|[Gertie Potter, T...|
|tt0000653-A Calam...|[John R. Cumpson,...|
|tt0000841-The Day...|[Marion Leonard, ...|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
# Caching aka persisting-> otherwise randomsplit will lead to inconsistencies, since it changes the randomsplit each
# time you perform an operation on the splitted data
moviesasbaskets_d2 = moviesasbaskets_d2.persist(pyspark.StorageLevel.MEMORY_AND_DISK)

In [ ]:
# apply the randomSplit() 
MBAtrain2,MBAtest2 = moviesasbaskets_d2.randomSplit([0.9, 0.1], seed=13)

In [ ]:
from pyspark.ml.fpm import FPGrowth
fpGrowth2 = FPGrowth(itemsCol="cast", minSupport=0.0001, minConfidence=0)
model_d2 = fpGrowth2.fit(MBAtrain2)  #excluding the episodes we consider frequent an actor if it appears in (distinct tconst = 974512)*0.0001 97 films 
#reasonable

In [ ]:
# Display frequent itemsets.-> NB this result includes all size items!
freqItemsets_d2 = model_d2.freqItemsets
freqItemsets_d2.createOrReplaceTempView("freqItemsets_d2")
freqItemsets_d2.show()

In [ ]:
from pyspark.sql.functions import col, asc,desc
freqItemsets_d2.orderBy(col('freq').desc()).show() 
#the most frequent actor is no more nm10120013 appearing in > 5000 films BUT Mel Blanc with 925 movies
#however taking as minSupport 0.0005 only 5 actors are retrieved and therefore no association rules
#therefore try again with a minSupport of 0.0001-> now it works

In [ ]:
# Display generated association rules.
associationRules_d2 = model_d2.associationRules
associationRules_d2.createOrReplaceTempView("associationRules_d2")
associationRules_d2.show(n=5) 

In [ ]:
#now only two rows in association rules having confidence equal to 1
spark.sql("""select (*)
           from associationRules_d2
           where confidence = '1.0' 
           limit 20""").show()
#We obtain that where there are Curly Howard and Larry Fine there is Moe Howard and where there are Curly Howard and Moe Howard there is Larry Fine

In [ ]:
#for a better understanding ###INTERESTING
spark.sql("""select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence
              from associationRules_d2
              order by confidence desc
              limit 20""").show()

In [ ]:
# transform examines the input items against all the association rules and summarize the consequents as prediction
# it would be useful to divide the db in training and test set in order to actually do predictions
association_d2 = model_d2.transform(MBAtest2)
associations_d2.show()

In [ ]:
#Doing predictions for MBAtest2
associations_d2.createOrReplaceTempView('associations_d2')
pred2= spark.sql("""select *
                      from associations_d2
                      where size(prediction) >= 1 """)
pred2.createOrReplaceTempView('pred2')
pred2.show() #718 rows in pred_ex2

In [ ]:
#focus the analysis for frequent itemset with one actor only (frequent actors)
#NOW filter freqItemsets by those having items = 1 -> most frequent actors banally
mostfrequentactor_d2= spark.sql("""select items, freq
                                    from freqItemsets_d2
                                    where size(items) = 1
                                    order by freq desc""")
mostfrequentactor_d2.show(n=5)
mostfrequentactor_d2.count()

In [ ]:
#now analysis for frequent itemset with two actors
mostfrequent2actors_d2= spark.sql("""select items, freq
                                      from freqItemsets_d2
                                      where size(items) = 2
                                      order by freq desc""")
mostfrequent2actors_d2.show(n=5)
mostfrequent2actors_d2.count()

In [ ]:
#now analysis for frequent itemset with three actors
mostfrequent3actors_d2= spark.sql("""select items, freq
                                      from freqItemsets_d2
                                      where size(items) = 3
                                      order by freq desc""")
mostfrequent3actors_d2.show(n=5)
mostfrequent3actors_d2.count() 

In [ ]:
#for more than 3 actors there is only an itemset, containing 4 actors, considered frequent according to our parameter and having freq = 88
#now analysis for frequent itemset with three actors
mostfrequentnactors_d2= spark.sql("""select items, freq
                                      from freqItemsets_d2
                                      where size(items) > 3
                                      order by freq desc""")
mostfrequentnactors_d2.show(n=5)
mostfrequentnactors_d2.count()

In [ ]:
mostfrequentnactors_d2.toPandas()['items'].tolist() 
#just to visualize that the 4 most frequent actors are [['Sammy Brooks', 'Harold Lloyd', 'Bebe Daniels', "'Snub' Pollard"]]
#appearing together in 88 films